Check GPU availability

In [13]:
import torch
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))

True
Tesla T4


Set up working environment

In [6]:
# Mount Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
drive_data_dir = "/content/drive/MyDrive/images_to_train/tiff"

In [1]:

# Clone/update your repository
! rm -rf /content/Image_analysis
! git clone https://github.com/Nerita21/Image_analysis.git



Cloning into 'Image_analysis'...
remote: Enumerating objects: 89, done.
remote: Counting objects: 100% (89/89), done.
remote: Compressing objects: 100% (62/62), done.
remote: Total 89 (delta 32), reused 69 (delta 19), pack-reused 0 (from 0)
Receiving objects: 100% (89/89), 19.82 KiB | 2.83 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [2]:
# Install packages from requirements only if missing
import subprocess
import sys
import importlib.util
import os

def ensure_requirements(req_file="requirements.txt"):
    if not os.path.exists(req_file):
        # Try to find requirements.txt in the cloned repository directory
        repo_dir = "/content/Image_analysis"
        if os.path.exists(os.path.join(repo_dir, req_file)):
            req_file = os.path.join(repo_dir, req_file)
        else:
            print(f"Error: requirements.txt not found at {req_file} or {os.path.join(repo_dir, 'requirements.txt')}")
            return

    with open(req_file) as f:
        for line in f:
            pkg = line.strip()
            if not pkg or pkg.startswith("#"):
                continue

            # Pip typically expects '==' for exact version pinning.
            # If the requirement uses '=', try to convert it to '=='
            # if it looks like a simple package=version string and not a URL or path.
            processed_pkg = pkg
            if '=' in pkg and '==' not in pkg and not any(x in pkg for x in ['/', 'git+', '#egg=']):
                parts = pkg.split('=', 1)
                if len(parts) == 2 and parts[1]: # ensure there's a version part
                    processed_pkg = f"{parts[0]}=={parts[1]}"

            name = pkg.split("==")[0].split(">=")[0].split("<=")[0].split("=")[0]
            if importlib.util.find_spec(name) is None:
                print(f"Installing {processed_pkg}...")
                subprocess.check_call([sys.executable, "-m", "pip", "install", processed_pkg])
            else:
                print(f"{name} already installed.")

ensure_requirements()


Installing cellpose==4.0.4...
Installing napari==0.6.5...
tifffile already installed.
Installing nd2==0.10.3...
Installing scikit-image==0.25.2...
Installing scikit-learn==1.7.2...
scipy already installed.
seaborn already installed.
yaml already installed.


In [3]:
# Import required packages:
from cellpose import models, utils
import napari
import matplotlib.pyplot as plt
from skimage import io
import os
import tifffile
import numpy as np



Welcome to CellposeSAM, cellpose v
cellpose version: 	4.0.4 
platform:       	linux 
python version: 	3.12.12 
torch version:  	2.9.0+cu126! The neural network component of
CPSAM is much larger than in previous versions and CPU excution is slow. 
We encourage users to use GPU/MPS if available. 




In [8]:
# Now you can import your packages
import sys
sys.path.insert(0, '/content/Image_analysis/src')   # adjust path if you cloned elsewhere
from utils import load_config
config, base_dir = load_config()

from package import (convert_nd2_to_tiff,
                     view_tiff_directory,
                     view_masks_napari,
                     create_masks)

In [9]:
# Load paths
raw_image_dir = base_dir / config["data"]["raw_images_dir"]
raw_tiff_image_dir = base_dir / config["data"]["raw_tiff_images_dir"]
raw_tiff_image_dir_train =  os.listdir(drive_data_dir)


Convert and examine files (optional)

In [ ]:
# Convert ND2 to TIFF (optional)
convert_nd2_to_tiff(raw_image_dir, raw_tiff_image_dir)


In [ ]:

# Load images into Napari for viewing
view_tiff_directory(raw_tiff_image_dir_train, group_channels=True)


Load images for segmentation

In [10]:
from pathlib import Path

# Load all images
# Ensure raw_tiff_image_dir_train is a Path object if you want to use .glob()
drive_data_path = Path(drive_data_dir)
cyto_img_path = sorted(drive_data_path.glob("*ch1.tiff"))
nuc_img_path = sorted(drive_data_path.glob("*ch2.tiff"))

cyto_imgs = [io.imread(p) for p in cyto_img_path]
nuc_imgs = [io.imread(p) for p in nuc_img_path]


Create masks with cellpose

In [11]:
# Call Cellpose model and create raw masks for cytoplasm
model = models.CellposeModel(pretrained_model='cpsam', gpu=True)
# Create raw masks for cytoplasm
create_masks(drive_data_path, model)

100%|██████████| 1.15G/1.15G [00:06<00:00, 188MB/s]


Processing H1792_NEAT1_40x_v2_ch1.tiff...
Saved mask to H1792_NEAT1_40x_v2_ch1_mask.tiff
Processing H1792_NEAT1_40x_v2_ch2.tiff...
Saved mask to H1792_NEAT1_40x_v2_ch2_mask.tiff
Processing H1792_PVT1_digitonin_v2_ch1.tiff...
Saved mask to H1792_PVT1_digitonin_v2_ch1_mask.tiff
Processing H1792_PVT1_digitonin_v2_ch2.tiff...
Saved mask to H1792_PVT1_digitonin_v2_ch2_mask.tiff
Processing H1792_PVT1_v2_ch1.tiff...
Saved mask to H1792_PVT1_v2_ch1_mask.tiff
Processing H1792_PVT1_v2_ch2.tiff...
Saved mask to H1792_PVT1_v2_ch2_mask.tiff
Processing H1792_PVT1ex_01tri_50for_v2_ch1.tiff...
Saved mask to H1792_PVT1ex_01tri_50for_v2_ch1_mask.tiff
Processing H1792_PVT1ex_01tri_50for_v2_ch2.tiff...
Saved mask to H1792_PVT1ex_01tri_50for_v2_ch2_mask.tiff
Processing H1792_PVT1ex_05tri_50for_v2_ch1.tiff...
Saved mask to H1792_PVT1ex_05tri_50for_v2_ch1_mask.tiff
Processing H1792_PVT1ex_05tri_50for_v2_ch2.tiff...
Saved mask to H1792_PVT1ex_05tri_50for_v2_ch2_mask.tiff
Processing H1792_PVT1ex_digitonin_v2_c

Corrects masks in napari and save corrected (outside colab). Run napari_examiner.py to open masks and images.